In [1]:
from androguard.misc import *
import os
import gc
import json
import re
import subprocess
environment_constant = {'APKS_FOLDER':"apks", 'OUT_FOLDER':"out", "RULE_PATH": "rules"}

In [2]:
class Node:
    total_node = 0

    def __init__(self, class_name):
        self.class_name = class_name
        self.methods = []
        Node.total_node +=1

    def addMethod(self, method):
        if method not in self.methods:
            self.methods.append(method)

In [3]:
def extract_jsinterface(APK_NAME):
    androguard_apk_obj, androguard_d_array,androguard_dx = AnalyzeAPK(environment_constant['APKS_FOLDER']+'/'+APK_NAME, session = None)
    EncodedMethodList = []
    MethodAnalysisList = []
    for dvm in androguard_d_array:
        for adi in dvm.map_list.get_item_type("TYPE_ANNOTATIONS_DIRECTORY_ITEM"):
            if adi.get_method_annotations() == []:
                continue
            for mi in adi.get_method_annotations():
                ann_set_item = dvm.CM.get_obj_by_offset(mi.get_annotations_off())
                for aoffitem in ann_set_item.get_annotation_off_item():
                    annotation_item = dvm.CM.get_obj_by_offset(aoffitem.get_annotation_off())
                    encoded_annotation = annotation_item.get_annotation()
                    if "Landroid/webkit/JavascriptInterface" in str(dvm.CM.get_type(encoded_annotation.get_type_idx())):
                        #print(type(dvm.get_method_by_idx(mi.get_method_idx())))
                        EncodedMethodList.append(dvm.get_method_by_idx(mi.get_method_idx()))


    interface_and_method = list()

    for encoded_method in EncodedMethodList:
            class_name = encoded_method.get_class_name()
            
            class_exist = False

            for node in interface_and_method:
                if class_name == node.class_name:
                    node.addMethod(encoded_method)
                    class_exist = True
                    break

            if not class_exist:
                temp = Node(class_name)
                temp.addMethod(encoded_method)
                interface_and_method.append(temp)

    return interface_and_method

In [4]:
def convert_signature(signature):
        # Java 형식의 시그니처에서 '/'를 '.'으로 대체하고, L과 ;를 제거
        return signature.replace('/', '.')[1:-1]
def parse_and_convert_method_signature(signature_string):

    # 정규표현식을 사용하여 파라미터와 반환 변수 추출
    pattern = r'\((.*?)\)(.*)'
    match = re.match(pattern, signature_string)
    
    if match:
        parameters = match.group(1).split(',')
        parameters = [convert_signature(param.strip()) for param in parameters if param.strip()]  # 각 파라미터에 convert_signature 적용
        return_type = convert_signature(match.group(2).strip())
        
        return parameters, return_type
    else:
        return None, None


In [5]:
def write_template(class_name, method,APK_NAME):
    parameters, return_type = parse_and_convert_method_signature(method.get_descriptor())
    if(not(return_type)):
        return_type = "void"
    # 주어진 JSON 데이터
    json_data = {
        f"{method.get_name()}": {
            "enable": True,
            "SliceMode": True,
            "traceDepth": 6,
            "desc": {
                "name": f"{method.get_name()}",
                "category": "interface_analysis",
                "detail": "identify if it's a vulnerable Javascript interface",
                "class_name": f"{convert_signature(class_name)}"
            },
            "entry": {},
            "source": {
                "Param": {
                    f"<{convert_signature(class_name)}: {return_type} {method.get_name()}(*)>": [
                        "p*"
                    ]
                }
            },
            "sink": {
                "<*: * loadUrl*(*)>": {
                    "TaintCheck": [
                        "p*"
                    ]
                }
            }
        }
    }

    # JSON 파일에 쓰기
    current_directory = os.getcwd()

    rule_folder_path = os.path.join(current_directory,environment_constant['OUT_FOLDER'], APK_NAME, environment_constant['RULE_PATH'])

    with open(f'{rule_folder_path}/{method.get_name()}.json', 'w') as json_file:
        json.dump(json_data, json_file, indent=2)  # indent 옵션을 사용하여 들여쓰기를 설정할 수 있습니다.


In [6]:
def make_analysis_template(interface_and_method,APK_NAME):
    for node in interface_and_method:
        print('[class]:', node.class_name)
        for method in node.methods:
            print('\t ->', method.get_name())
            print('\t\t->', method.get_descriptor())
            write_template(node.class_name, method,APK_NAME)

In [7]:
def create_json_config(APK_NAME):

    current_directory = os.getcwd()
    apk_path = os.path.join(current_directory,environment_constant['APKS_FOLDER'],APK_NAME + '.apk')
    rule_path = os.path.join(current_directory, environment_constant['OUT_FOLDER'],APK_NAME, environment_constant['RULE_PATH'])
    out_path = os.path.join(current_directory,environment_constant['OUT_FOLDER'],APK_NAME)

    json_content = {
        "apkPath": apk_path,
        "rulePath": rule_path,
        "javaSource": True,
        "out": out_path
        # 여기에 필요한 다른 속성 추가 가능
    }

    json_file_path = os.path.join(out_path, f'{APK_NAME}_config.json')

    

    print(f'[*] {json_file_path} 생성')
    with open(json_file_path, 'w') as json_file:    
        json.dump(json_content, json_file, indent=2)

    return json_file_path

In [8]:
def run_appshark(config_file_path):
    command = f'java -jar build/libs/AppShark-0.1.2-all.jar {config_file_path}'

    subprocess.run(command,shell=True)

In [9]:
def make_structure(APK_NAME):
    current_directory = os.getcwd()

    output_folder_path = os.path.join(current_directory,environment_constant['OUT_FOLDER'])

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
        print(f"[*]'{environment_constant['OUT_FOLDER']}' 폴더가 생성되었습니다.")

    
    apk_folder_path = os.path.join(output_folder_path, APK_NAME)
    if not os.path.exists(apk_folder_path):
        os.makedirs(apk_folder_path)
        print(f"[*]'{environment_constant['OUT_FOLDER']}/{APK_NAME}' 폴더가 생성되었습니다.")
    
    rule_folder_path = os.path.join(apk_folder_path, environment_constant['RULE_PATH'])

    if not os.path.exists(rule_folder_path):
        os.makedirs(rule_folder_path)
        print(f"[*]'{environment_constant['OUT_FOLDER']}/{APK_NAME}/{environment_constant['RULE_PATH']}' 폴더가 생성되었습니다.")


In [10]:
def main():
    TEMP_APK = 'com.hyundai.myhyundai.apk'

    make_structure(TEMP_APK)
    
    extracted_JSinterface = extract_jsinterface(TEMP_APK)
    make_analysis_template(extracted_JSinterface,TEMP_APK)

    config_file_path = create_json_config(TEMP_APK)

    run_appshark(config_file_path)

In [58]:
def make_result(APK_NAME):
    current_directory = os.getcwd()

    result_path = os.path.join(current_directory,environment_constant['OUT_FOLDER'],APK_NAME,'results.json')
    vuln_list_path= os.path.join(current_directory,environment_constant['OUT_FOLDER'],APK_NAME,'vuln_list.txt')

    with open(result_path, 'r') as file:
        json_data = json.load(file)

    with open(vuln_list_path,'w') as f:
        f.write(f'[APK_NAME]: {APK_NAME}\n')
        f.write(f'[Vuln Method List]\n')

        for method in json_data['SecurityInfo']['interface_analysis']:
            f.write(f"[Name]:{method}\n\n")
            f.write(f"\t[Source]:{json_data['SecurityInfo']['interface_analysis'][method]['vulners'][0]['details']['Source']}\n")
            f.write(f"\t[Sink]:{json_data['SecurityInfo']['interface_analysis'][method]['vulners'][0]['details']['Sink']}\n")
            
            

In [11]:
if __name__ == "__main__":
    main()

Requested API level 33 is larger than maximum we have, returning API level 28 instead.


[class]: Lcom/hyundai/myhyundai/common/MyhyundaiWVBridge;
	 -> getDeviceId
		-> ()Ljava/lang/String;
	 -> getDeviceToken
		-> ()Ljava/lang/String;
	 -> postMessage
		-> (Ljava/lang/String;)V
	 -> printLog
		-> (Ljava/lang/String;)V
	 -> showToast
		-> (Ljava/lang/String;)Ljava/lang/String;
[class]: Lcom/hyundai/myhyundai/common/GAWebAppInterface;
	 -> GA_DATA
		-> (Ljava/lang/String;)V
[class]: Lcom/hyundai/myhyundai/activity/MainActivity$bwcBridge;
	 -> setMessage
		-> (Ljava/lang/String;)V
[*] c:\android\analyze_interface\out\com.hyundai.myhyundai.apk\com.hyundai.myhyundai.apk_config.json 생성
